### PDF文档解析

In [1]:
import re
import pandas as pd
from typing import List, Union
from pathlib import Path
from tqdm import tqdm
from src.modules.document.pdf_parser import PdfParserUsingPyMuPDF


def cleanup(document: List[str]):
    """
    清洗数据 
    """
    # '\n', ' ' 删除
    text = text.replace('\n','').replace(' ','')
        
    
    # 删除本文档为2024CCFBDC***
    head_pattern = '本文档为2024CCFBDCI比赛用语料的一部分。[^\s]+仅允许在本次比赛中使用。'
    
    # news_pattern
    pattern1 = r"发布时间：[^\s]+发布人：新闻宣传中心"
    pattern2 = r"发布时间：[^\s]+发布人：新闻发布人"
    pattern3 =  r'发布时间：\d{4}年\d{1,2}月\d{1,2}日'
    news_pattern = head_pattern+'|'+pattern1+'|'+pattern2+'|'+pattern3
    text = re.sub(news_pattern,'',text)

    # report_pattern
    report_pattern1 = '第一节重要提示[^\s]+本次利润分配方案尚需提交本公司股东大会审议。'
    report_pattern12 = '一重要提示[^\s]+股东大会审议。'
    report_pattern13 = '一、重要提示[^\s]+季度报告未经审计。'
    report_pattern2 = '本公司董事会及全体董事保证本公告内容不存在任何虚假记载、[^\s]+季度财务报表是否经审计□是√否'
    report_pattern3 = '中国联合网络通信股份有限公司（简称“公司”）董事会审计委员会根据相关法律法规、[^\s]+汇报如下：'
    report_pattern = report_pattern1+'|'+report_pattern12+'|'+report_pattern13+'|'+report_pattern2+'|'+report_pattern3
    text = re.sub( report_pattern,'',text)
#     white paper 版本一 效果不好
    # 优先级别 bp1 bp2 bp3
#     bp_pattern_law = '版权声明[^\s]+追究其相关法律责任。'
#     bp_pattern1 = r'目录.*?披露发展报告（\d{4}年）' # 只针对AZ08.pdf
#     bp_pattern2 = r'目录.*?白皮书.*?（\d{4}年）'
#     bp_pattern3 = r'目录.*?白皮书'
#     bp_pattern = bp_pattern_law  +'|'+bp_pattern1+'|'+bp_pattern2+'|'+bp_pattern3
#     text = re.sub(bp_pattern,'',text)
#     print(text)
    
    return text


__news__ = ["AF", "AT", "AW"]
__report__ = ["AZ", "AY"]

data = []
pdf_parser = PdfParserUsingPyMuPDF()
files: List[Path] = list(Path("dataset/A榜/A_document/").rglob("*.pdf"))
for i, pdf_file in enumerate(files):
    code = pdf_file.name[:2]
    if code in __news__:
        doc = pdf_parser.parse(pdf_file)
        cleaned = cleanup_news(pdf_file.name, doc)
        data.append(cleaned)
        print(pdf_file, cleaned.get("发布时间"), cleaned.get("标题"), len(cleaned.get("内容")))

df = pd.DataFrame(data)
print(df.head())
df.to_csv("文档结构化.csv", index=False)

南方暴雨来袭中国联通全力保障通信网络畅通


NameError: name 'cleanup_news' is not defined

In [2]:
def cleanup_news(fnm: str, document: Union[str, List[str]]) -> dict:
    """
    清洗新闻
    """
    if isinstance(document, list):
        document = "".join(document)
    # 使用正则表达式提取标题、发布时间、发布人和内容
    title_match = re.search(r'本文档.*?仅允许在本次比赛中使用。\n(.*?)\n', document, re.MULTILINE | re.DOTALL)
    pub_time_match = re.search(r'(发布时间)：(\d{4}-\d{2}-\d{2}|\d{4} 年\d{1,2} 月\d{1,2} 日)', document, re.MULTILINE | re.DOTALL)
    content_match = re.search(r'(\d{4}-\d{2}-\d{2}|\d{4} 年\d{1,2} 月\d{1,2} 日)(.*)', document, re.MULTILINE | re.DOTALL)
    author_match = re.search(r'(发布人|文章来源)：(.*?)(?=\n)', document, re.MULTILINE | re.DOTALL)
    author_match_delete = re.search(r'((发布人|文章来源)：.*?(?=\n))', document, re.MULTILINE | re.DOTALL)
    
    # 提取数据
    title = title_match.group(1).strip() if title_match else ""
    pub_time = pub_time_match.group(1).strip() if pub_time_match else ""
    content = content_match.group(2).strip() if content_match else ""
    author = author_match.group(2).strip() if author_match else ""
    author_match = author_match_delete.group(1).strip() if author_match else ""
    
    # 结果字典
    return {
        "文件名": fnm,
        "标题": title.replace("\n", "").replace(" ", ""),
        "内容": content.replace("\n", "").replace(" ", "").replace(author_match, ""),
        "发布时间": pub_time.replace("\n", "").replace(" ", ""),
        "发布人": author.replace("\n", "").replace(" ", "")
    }

# f = Path("dataset/A榜/A_document/AW09.pdf")
f = Path("dataset/A榜/A_document/AW10.pdf")
TEMP_NEWS_TEXT = "".join(PdfParserUsingPyMuPDF().parse(f))
print([TEMP_NEWS_TEXT])
cleanup_news(f.name, TEMP_NEWS_TEXT)


['本文档为2024 CCF BDCI 比赛用语料的一部分。部分文档使用大语言模型改写生成，内容可能与现实情况\n不符，可能不具备现实意义，仅允许在本次比赛中使用。\n以数字化网络化智能化助力中国式现代化\n——2022 年中国联通合作伙伴大会主旨演讲\n（2022 年12 月20 日）\n刘烈宏\n尊敬的各位嘉宾、线上的朋友们：\n大家好！非常高兴与大家相聚在2022 年中国联通合作伙伴大会。首先，我代表中国联\n通，向大家长期以来给予我们的关心和支持，表示衷心的感谢！\n党的二十大擘画了以中国式现代化全面推进中华民族伟大复兴的宏伟蓝图，作出了“加\n快建设网络强国、数字中国”“加快发展数字经济”的战略部署，照亮了信息通信业新一轮高质\n量发展的光明前景！一方面，中国式现代化的宏伟进程必将进一步加速推动数字化网络化智\n能化在更广领域实现更高质量、更深层次的发展；另一方面，数字化网络化智能化正成为实\n现中国式现代化重组要素资源、重塑经济结构、改变竞争格局的关键力量。面对中国式现代\n化与新一代数字革命浪潮同频共振带来的战略机遇，我们真诚希望与各界一起，踔厉奋发，\n勇毅前行，共同打好数字化网络化智能化高质量发展的“团体赛”，共同奋进中国式现代化的“新\n征程”。\n一年来，中国联通坚持以“数字信息基础设施运营服务国家队、网络强国数字中国智慧\n社会建设主力军、数字技术融合创新排头兵”的责任担当，在“强基固本、守正创新、融合开\n放”的战略指引下，大力发展“大联接、大计算、大数据、大应用、大安全”五大主责主业，\n坚定奋楫数字经济主航道，全面推进“1+9+3”战略规划体系见行见效。公司发展质量持续向\n好，营业收入和净利润增幅均创近年来新高；发展结构不断优化，产业数字化业务成为收入\n增长“第一引擎”，对公司增量收入贡献超过70%；发展基础更加稳固，面向数字经济新需求\n的网络、产品和服务供给水平不断提升；发展优势更加凸显，“一个联通、一体化能力聚合、\n一体化运营服务”的特色能力体系正在积厚成势。总的来讲，9 个行动计划和3 个指导意见\n都取得了阶段性成效，形成了标志性成果：\n一是新型数字信息基础设施行动计划成效显著，网络品质迈上新台阶。我们携手中国电\n信累计开通5G 共享基站100 万站，实现乡镇及以上区域5G 网络连续覆盖，速率更快、可\n靠性更高、场

{'文件名': 'AW10.pdf',
 '标题': '以数字化网络化智能化助力中国式现代化',
 '内容': '）刘烈宏尊敬的各位嘉宾、线上的朋友们：大家好！非常高兴与大家相聚在2022年中国联通合作伙伴大会。首先，我代表中国联通，向大家长期以来给予我们的关心和支持，表示衷心的感谢！党的二十大擘画了以中国式现代化全面推进中华民族伟大复兴的宏伟蓝图，作出了“加快建设网络强国、数字中国”“加快发展数字经济”的战略部署，照亮了信息通信业新一轮高质量发展的光明前景！一方面，中国式现代化的宏伟进程必将进一步加速推动数字化网络化智能化在更广领域实现更高质量、更深层次的发展；另一方面，数字化网络化智能化正成为实现中国式现代化重组要素资源、重塑经济结构、改变竞争格局的关键力量。面对中国式现代化与新一代数字革命浪潮同频共振带来的战略机遇，我们真诚希望与各界一起，踔厉奋发，勇毅前行，共同打好数字化网络化智能化高质量发展的“团体赛”，共同奋进中国式现代化的“新征程”。一年来，中国联通坚持以“数字信息基础设施运营服务国家队、网络强国数字中国智慧社会建设主力军、数字技术融合创新排头兵”的责任担当，在“强基固本、守正创新、融合开放”的战略指引下，大力发展“大联接、大计算、大数据、大应用、大安全”五大主责主业，坚定奋楫数字经济主航道，全面推进“1+9+3”战略规划体系见行见效。公司发展质量持续向好，营业收入和净利润增幅均创近年来新高；发展结构不断优化，产业数字化业务成为收入增长“第一引擎”，对公司增量收入贡献超过70%；发展基础更加稳固，面向数字经济新需求的网络、产品和服务供给水平不断提升；发展优势更加凸显，“一个联通、一体化能力聚合、一体化运营服务”的特色能力体系正在积厚成势。总的来讲，9个行动计划和3个指导意见都取得了阶段性成效，形成了标志性成果：一是新型数字信息基础设施行动计划成效显著，网络品质迈上新台阶。我们携手中国电信累计开通5G共享基站100万站，实现乡镇及以上区域5G网络连续覆盖，速率更快、可靠性更高、场景也更丰富，我们还进一步深化4G共建共享，累计开通4G共享基站110万站。通过5G/4G共建共享，双方累计为国家节省投资超过2700亿元，节约运营成本每年超过300亿元，减少碳排放每年超1000万吨。特别是我们加快推进900MHz低频打底网建设，今年投资200亿元，快速新建了17